# Todo
1. use [center loss](https://medium.com/mlreview/experiments-with-a-new-loss-term-added-to-the-standard-cross-entropy-85b080c42446)
2. split train & val
3. add [Beam Search](https://machinelearningmastery.com/beam-search-decoder-natural-language-processing/)
4. maybe add [attention](https://towardsdatascience.com/light-on-math-ml-attention-with-keras-dc8dbc1fad39)
5. add [reg](https://machinelearningmastery.com/how-to-reduce-overfitting-with-dropout-regularization-in-keras/)

In [7]:
import warnings
warnings.filterwarnings("ignore")
import src.dataset as ds
import numpy as np
from src.embeddings import extract_embedding_weights
from keras.layers import Embedding, CuDNNLSTM, Bidirectional, Dense, CuDNNGRU
from keras.initializers import Constant
from keras import Sequential
from tensorflow.python.keras.callbacks import TensorBoard
from src.model import Model
from keras.preprocessing.sequence import pad_sequences

from time import time
import tensorflow as tf

SEQUENCE_LEN = 1000

In [2]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

In [3]:
X, y, tokenizer = ds.load_tokenized_data()
embedding_matrix = extract_embedding_weights()

100%|███████████████████████████████████████████████████████████████████████████████| 615/615 [00:02<00:00, 249.36it/s]


In [9]:
X_padded = pad_sequences([[sample] for sample in X], maxlen=SEQUENCE_LEN)

In [10]:
X_padded

array([[   0,    0,    0, ...,    0,    0, 6908],
       [   0,    0,    0, ...,    0,    0, 3040],
       [   0,    0,    0, ...,    0,    0, 1840],
       ...,
       [   0,    0,    0, ...,    0,    0, 2824],
       [   0,    0,    0, ...,    0,    0, 4884],
       [   0,    0,    0, ...,    0,    0, 5990]])

In [6]:
model = Model(tokenizer, embedding_matrix)
model.train(X_padded,y, epochs=1, batch_size=20, callbacks=[tensorboard])
# model.test()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 300)            2251800   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100)               140400    
_________________________________________________________________
dense_1 (Dense)              (None, 7506)              758106    
Total params: 3,150,306
Trainable params: 898,506
Non-trainable params: 2,251,800
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 154596 samples, validate on 38650 samples
Epoch 1/1


  6440/154596 [>.............................] - ETA: 4:52:56 - loss: 8.9222 - _perplexity: 485.12 - ETA: 1:40:19 - loss: 8.9201 - _perplexity: 484.43 - ETA: 1:01:38 - loss: 8.9157 - _perplexity: 482.95 - ETA: 45:13 - loss: 8.9121 - _perplexity: 481.7673 - ETA: 36:12 - loss: 8.9063 - _perplexity: 479.89 - ETA: 30:23 - loss: 8.9043 - _perplexity: 479.23 - ETA: 26:23 - loss: 8.8994 - _perplexity: 477.66 - ETA: 23:24 - loss: 8.8922 - _perplexity: 475.41 - ETA: 21:05 - loss: 8.8880 - _perplexity: 474.11 - ETA: 19:17 - loss: 8.8699 - _perplexity: 468.65 - ETA: 17:50 - loss: 8.8612 - _perplexity: 466.07 - ETA: 16:38 - loss: 8.8537 - _perplexity: 463.91 - ETA: 15:36 - loss: 8.8393 - _perplexity: 460.04 - ETA: 14:45 - loss: 8.8200 - _perplexity: 454.89 - ETA: 14:04 - loss: 8.7978 - _perplexity: 449.27 - ETA: 13:26 - loss: 8.7833 - _perplexity: 445.68 - ETA: 12:52 - loss: 8.7615 - _perplexity: 440.38 - ETA: 12:20 - loss: 8.7360 - _perplexity: 434.86 - ETA: 11:52 - loss: 8.7145 - _perplexity: 43

 12900/154596 [=>............................] - ETA: 4:47 - loss: 6.4941 - _perplexity: 414.494 - ETA: 4:47 - loss: 6.4916 - _perplexity: 415.811 - ETA: 4:47 - loss: 6.4852 - _perplexity: 414.663 - ETA: 4:47 - loss: 6.4826 - _perplexity: 414.359 - ETA: 4:46 - loss: 6.4783 - _perplexity: 414.743 - ETA: 4:46 - loss: 6.4689 - _perplexity: 412.749 - ETA: 4:46 - loss: 6.4640 - _perplexity: 412.405 - ETA: 4:45 - loss: 6.4623 - _perplexity: 412.429 - ETA: 4:45 - loss: 6.4562 - _perplexity: 411.549 - ETA: 4:45 - loss: 6.4497 - _perplexity: 411.478 - ETA: 4:44 - loss: 6.4411 - _perplexity: 410.314 - ETA: 4:44 - loss: 6.4384 - _perplexity: 410.051 - ETA: 4:44 - loss: 6.4415 - _perplexity: 413.654 - ETA: 4:43 - loss: 6.4355 - _perplexity: 413.864 - ETA: 4:43 - loss: 6.4340 - _perplexity: 415.452 - ETA: 4:43 - loss: 6.4355 - _perplexity: 417.508 - ETA: 4:42 - loss: 6.4390 - _perplexity: 419.443 - ETA: 4:42 - loss: 6.4334 - _perplexity: 419.583 - ETA: 4:42 - loss: 6.4298 - _perplexity: 418.743 - E

 19400/154596 [==>...........................] - ETA: 4:13 - loss: 6.1067 - _perplexity: 434.969 - ETA: 4:12 - loss: 6.1049 - _perplexity: 435.175 - ETA: 4:13 - loss: 6.1019 - _perplexity: 434.207 - ETA: 4:13 - loss: 6.1015 - _perplexity: 434.622 - ETA: 4:13 - loss: 6.0988 - _perplexity: 434.213 - ETA: 4:13 - loss: 6.0960 - _perplexity: 433.863 - ETA: 4:12 - loss: 6.0950 - _perplexity: 434.327 - ETA: 4:12 - loss: 6.0951 - _perplexity: 434.498 - ETA: 4:12 - loss: 6.0952 - _perplexity: 434.958 - ETA: 4:12 - loss: 6.0925 - _perplexity: 434.693 - ETA: 4:12 - loss: 6.0918 - _perplexity: 436.414 - ETA: 4:12 - loss: 6.0905 - _perplexity: 435.940 - ETA: 4:12 - loss: 6.0885 - _perplexity: 435.454 - ETA: 4:12 - loss: 6.0851 - _perplexity: 434.550 - ETA: 4:12 - loss: 6.0831 - _perplexity: 434.277 - ETA: 4:11 - loss: 6.0818 - _perplexity: 433.613 - ETA: 4:11 - loss: 6.0813 - _perplexity: 434.063 - ETA: 4:11 - loss: 6.0825 - _perplexity: 436.577 - ETA: 4:11 - loss: 6.0821 - _perplexity: 437.318 - E

 25860/154596 [====>.........................] - ETA: 3:53 - loss: 5.9315 - _perplexity: 463.296 - ETA: 3:53 - loss: 5.9302 - _perplexity: 463.008 - ETA: 3:53 - loss: 5.9286 - _perplexity: 462.688 - ETA: 3:53 - loss: 5.9273 - _perplexity: 462.338 - ETA: 3:53 - loss: 5.9267 - _perplexity: 466.500 - ETA: 3:53 - loss: 5.9256 - _perplexity: 466.521 - ETA: 3:53 - loss: 5.9256 - _perplexity: 465.973 - ETA: 3:53 - loss: 5.9246 - _perplexity: 465.692 - ETA: 3:52 - loss: 5.9234 - _perplexity: 466.805 - ETA: 3:52 - loss: 5.9220 - _perplexity: 466.594 - ETA: 3:52 - loss: 5.9209 - _perplexity: 466.716 - ETA: 3:52 - loss: 5.9192 - _perplexity: 466.154 - ETA: 3:52 - loss: 5.9181 - _perplexity: 465.542 - ETA: 3:52 - loss: 5.9174 - _perplexity: 465.679 - ETA: 3:52 - loss: 5.9182 - _perplexity: 466.068 - ETA: 3:52 - loss: 5.9177 - _perplexity: 467.404 - ETA: 3:52 - loss: 5.9171 - _perplexity: 468.341 - ETA: 3:52 - loss: 5.9178 - _perplexity: 469.214 - ETA: 3:51 - loss: 5.9165 - _perplexity: 468.927 - E

...

110300/154596 [====================>.........] - ETA: 1:22 - loss: 5.2773 - _perplexity: 680.589 - ETA: 1:22 - loss: 5.2773 - _perplexity: 680.522 - ETA: 1:22 - loss: 5.2771 - _perplexity: 680.920 - ETA: 1:22 - loss: 5.2769 - _perplexity: 680.737 - ETA: 1:22 - loss: 5.2767 - _perplexity: 680.841 - ETA: 1:22 - loss: 5.2765 - _perplexity: 681.443 - ETA: 1:22 - loss: 5.2762 - _perplexity: 681.249 - ETA: 1:22 - loss: 5.2759 - _perplexity: 681.029 - ETA: 1:22 - loss: 5.2757 - _perplexity: 680.914 - ETA: 1:22 - loss: 5.2756 - _perplexity: 680.684 - ETA: 1:22 - loss: 5.2754 - _perplexity: 680.924 - ETA: 1:22 - loss: 5.2752 - _perplexity: 681.009 - ETA: 1:21 - loss: 5.2750 - _perplexity: 680.896 - ETA: 1:21 - loss: 5.2750 - _perplexity: 681.199 - ETA: 1:21 - loss: 5.2745 - _perplexity: 681.095 - ETA: 1:21 - loss: 5.2743 - _perplexity: 680.895 - ETA: 1:21 - loss: 5.2742 - _perplexity: 680.670 - ETA: 1:21 - loss: 5.2740 - _perplexity: 680.576 - ETA: 1:21 - loss: 5.2740 - _perplexity: 680.492 - E

116540/154596 [=====================>........] - ETA: 1:12 - loss: 5.2531 - _perplexity: 690.171 - ETA: 1:12 - loss: 5.2530 - _perplexity: 689.980 - ETA: 1:12 - loss: 5.2528 - _perplexity: 689.778 - ETA: 1:11 - loss: 5.2527 - _perplexity: 690.197 - ETA: 1:11 - loss: 5.2523 - _perplexity: 690.214 - ETA: 1:11 - loss: 5.2523 - _perplexity: 690.172 - ETA: 1:11 - loss: 5.2522 - _perplexity: 689.968 - ETA: 1:11 - loss: 5.2522 - _perplexity: 690.334 - ETA: 1:11 - loss: 5.2519 - _perplexity: 690.110 - ETA: 1:11 - loss: 5.2519 - _perplexity: 689.964 - ETA: 1:11 - loss: 5.2516 - _perplexity: 690.102 - ETA: 1:11 - loss: 5.2516 - _perplexity: 690.309 - ETA: 1:11 - loss: 5.2512 - _perplexity: 690.103 - ETA: 1:11 - loss: 5.2512 - _perplexity: 689.979 - ETA: 1:11 - loss: 5.2512 - _perplexity: 690.199 - ETA: 1:11 - loss: 5.2510 - _perplexity: 689.992 - ETA: 1:11 - loss: 5.2508 - _perplexity: 689.799 - ETA: 1:11 - loss: 5.2507 - _perplexity: 689.629 - ETA: 1:10 - loss: 5.2504 - _perplexity: 689.409 - E

123160/154596 [======================>.......] - ETA: 1:02 - loss: 5.2314 - _perplexity: 698.871 - ETA: 1:02 - loss: 5.2312 - _perplexity: 698.941 - ETA: 1:02 - loss: 5.2310 - _perplexity: 698.754 - ETA: 1:02 - loss: 5.2312 - _perplexity: 698.953 - ETA: 1:02 - loss: 5.2311 - _perplexity: 698.797 - ETA: 1:02 - loss: 5.2309 - _perplexity: 698.627 - ETA: 1:01 - loss: 5.2310 - _perplexity: 698.483 - ETA: 1:01 - loss: 5.2309 - _perplexity: 698.976 - ETA: 1:01 - loss: 5.2306 - _perplexity: 699.012 - ETA: 1:01 - loss: 5.2304 - _perplexity: 698.836 - ETA: 1:01 - loss: 5.2304 - _perplexity: 699.076 - ETA: 1:01 - loss: 5.2307 - _perplexity: 699.511 - ETA: 1:01 - loss: 5.2308 - _perplexity: 700.547 - ETA: 1:01 - loss: 5.2307 - _perplexity: 700.738 - ETA: 1:01 - loss: 5.2308 - _perplexity: 700.779 - ETA: 1:01 - loss: 5.2307 - _perplexity: 700.695 - ETA: 1:01 - loss: 5.2304 - _perplexity: 700.532 - ETA: 1:01 - loss: 5.2303 - _perplexity: 700.597 - ETA: 1:01 - loss: 5.2304 - _perplexity: 700.552 - E

127980/154596 [=======================>......] - ETA: 51s - loss: 5.2138 - _perplexity: 712.31 - ETA: 51s - loss: 5.2137 - _perplexity: 712.15 - ETA: 51s - loss: 5.2137 - _perplexity: 712.11 - ETA: 51s - loss: 5.2137 - _perplexity: 712.51 - ETA: 51s - loss: 5.2135 - _perplexity: 712.33 - ETA: 51s - loss: 5.2132 - _perplexity: 712.18 - ETA: 51s - loss: 5.2131 - _perplexity: 712.04 - ETA: 51s - loss: 5.2130 - _perplexity: 712.71 - ETA: 51s - loss: 5.2128 - _perplexity: 712.77 - ETA: 51s - loss: 5.2127 - _perplexity: 712.61 - ETA: 51s - loss: 5.2129 - _perplexity: 712.65 - ETA: 51s - loss: 5.2129 - _perplexity: 712.56 - ETA: 51s - loss: 5.2130 - _perplexity: 713.08 - ETA: 50s - loss: 5.2128 - _perplexity: 713.09 - ETA: 50s - loss: 5.2127 - _perplexity: 712.98 - ETA: 50s - loss: 5.2126 - _perplexity: 712.95 - ETA: 50s - loss: 5.2127 - _perplexity: 713.57 - ETA: 50s - loss: 5.2125 - _perplexity: 713.40 - ETA: 50s - loss: 5.2126 - _perplexity: 713.81 - ETA: 50s - loss: 5.2126 - _perplexity: 

KeyboardInterrupt: 

In [ ]:
model.predict('Deep', 10)

> c:\users\eli\workspace\deep-learning-intro\assignment3\src\model.py(71)predict()
     70             ipdb.set_trace()
---> 71             out_word = ''
     72             for word, index in tokenizer.word_index.items():



ipdb>  proba


array([[2.6724436e-09, 2.8712122e-09, 1.7368726e-05, ..., 1.6459103e-06,
        5.8268895e-04, 2.2899949e-05]], dtype=float32)


ipdb>  proba.shape


(1, 7506)


...

[array([2.6724436e-09, 2.8712122e-09, 1.7368726e-05, ..., 1.6459103e-06,
       5.8268895e-04, 2.2899949e-05], dtype=float32)]


ipdb>  proba


array([[2.6724436e-09, 2.8712122e-09, 1.7368726e-05, ..., 1.6459103e-06,
        5.8268895e-04, 2.2899949e-05]], dtype=float32)


ipdb>  -np.sort(-proba)


array([[3.9417145e-01, 5.5460762e-02, 2.6946710e-02, ..., 2.4793119e-09,
        1.9104485e-09, 1.1865843e-09]], dtype=float32)


In [ ]:
# beam search
def beam_search_decoder(first_word, n_words, beam_width=3):
    tokenizer = self.tokenizer
    model = self.model
    in_text, result = first_word, first_word
    
	sequences = [[list(), 1.0]]
	# walk over each step in sequence
	for row in data:
		all_candidates = list()
		# expand each current candidate
		for i in range(len(sequences)):
			seq, score = sequences[i]
			for j in range(len(row)):
				candidate = [seq + [j], score * -log(row[j])]
				all_candidates.append(candidate)
		# order all candidates by score
		ordered = sorted(all_candidates, key=lambda tup:tup[1])
		# select k best
		sequences = ordered[:k]
	return sequences
 

In [ ]:
def beam_step(beam_sequences_scores):            
    all_candidates = []
    encoded = get_encoded(in_text, tokenizer)    
    for seq, score in beam_sequences_scores: # for each sequence
        seq.extend(encoded) # add words till now
        # predict top B words
        words_probs = model.predict_proba(encoded, verbose=0)
        words_probs_sorted = -np.sort(-words_probs) # sorting in descending order
        top_b_words_probs = words_probs_sorted[:B] # top B words with max probability
        # for each prob in top B words, create a candidate
        for prob in top_b_words_probs: 
            candidate = [seq + [word_token], score + np.log(prob + epsilon())] # todo: word_token 
            all_candidates.append(candidate)
    # take candidates with max score
    beam_sequences_scores = sorted(all_candidates, reverse=True, key=lambda seq, score: score)[:B]
    return beam_sequences_scores

In [ ]:
from keras.backend import epsilon
def predict(self, first_word, n_words, B=3):
        tokenizer = self.tokenizer
        model = self.model
        in_text, result = first_word, first_word
        beam_sequences_scores = [[list(), 0]]
        while len(result) < n_words: 
           
            for _ in range(B): # run till beam_width
                

                
                
        
def get_encoded(text, tokenizer):
    encoded = tokenizer.texts_to_sequences([text])[0]
    encoded = np.array(encoded)
    return encoded

In [ ]:
        
        
        # generate a fixed number of words
        for _ in range(n_words):
            # encode the text as integer
            encoded = tokenizer.texts_to_sequences([in_text])[0]
            encoded = np.array(encoded)
            # predict a word in the vocabulary
            proba = model.predict_proba(encoded, verbose=0)
            import ipdb
            ipdb.set_trace()
            out_word = ''
            for word, index in tokenizer.word_index.items():
                if index == yhat:
                    out_word = word
                    break
            # append to input
            in_text, result = out_word, result + ' ' + out_word
        return result